In [30]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [31]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier


In [32]:
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization

In [33]:
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [58]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [59]:
df = pd.read_csv('merged_Floor1.csv', index_col='Date', parse_dates=True)

In [60]:
dataset=pd.DataFrame(columns=['lights','acs','plug'])
dataset

,lights,acs,plug


In [61]:
dataset['lights']=df['z1_Light(kW)']+df['z2_Light(kW)']+df['z3_Light(kW)']+df['z4_Light(kW)']
dataset['acs']=df['z2_AC1(kW)']+df['z2_AC2(kW)']+df['z2_AC3(kW)']+df['z2_AC4(kW)']
dataset['plug']=df['z1_Plug(kW)']+df['z2_Plug(kW)']+df['z3_Plug(kW)']

In [62]:
dataset['Power_consumption']=dataset['lights']+dataset['acs']+dataset['plug']

In [63]:
dataset=dataset.fillna(method="ffill")

In [64]:
final_df=dataset

In [65]:
hour=final_df.resample('H').mean() #Hour wise consideration

In [66]:
hour

,lights,acs,plug,Power_consumption
Date,,,,
2018-07-01 00:00:00,73.675167,45.472833,36.477000,155.625000
2018-07-01 01:00:00,73.970167,45.849500,36.867333,156.687000
2018-07-01 02:00:00,73.782667,45.831833,36.668000,156.282500
2018-07-01 03:00:00,73.122167,45.616667,35.607667,154.346500
2018-07-01 04:00:00,83.146000,45.502833,33.766000,162.414833
...,...,...,...,...
2019-12-31 19:00:00,68.292167,23.921333,19.171833,111.385333
2019-12-31 20:00:00,68.485333,28.528000,19.147667,116.161000
2019-12-31 21:00:00,53.983000,22.726333,19.749833,96.459167


In [67]:
hour=np.array(hour)

In [68]:
n_step=1
lags=7

In [69]:
from numpy import array

def to_supervised(train,n_input,n_out):
    #falten data
    data=train
    X,y=list(),list()
    in_start=0
    for _ in range(len(data)):
        in_end=in_start+ n_input
        out_end=in_end + n_out
        if out_end<=len(data):
            x_input=data[ in_start:in_end,0]
            x_input=x_input.reshape((len(x_input)))
            X.append(x_input)
            y.append(data[in_end:out_end,0])
        in_start+=1
    return array(X), array(y) 

In [70]:
hour=np.array(hour)

In [71]:
X_hour,y_hour=to_supervised(hour,n_input=lags,n_out=n_step)

In [72]:
X_hour


array([[73.67516667, 73.97016667, 73.78266667, ..., 83.146     ,
        83.04066667, 56.90333333],
       [73.97016667, 73.78266667, 73.12216667, ..., 83.04066667,
        56.90333333, 55.00333333],
       [73.78266667, 73.12216667, 83.146     , ..., 56.90333333,
        55.00333333, 44.251     ],
       ...,
       [21.39483333, 20.63183333, 31.54833333, ..., 58.01316667,
        68.29216667, 68.48533333],
       [20.63183333, 31.54833333, 36.08283333, ..., 68.29216667,
        68.48533333, 53.983     ],
       [31.54833333, 36.08283333, 58.01316667, ..., 68.48533333,
        53.983     , 44.87083333]])

In [73]:
y_hour

array([[55.00333333],
       [44.251     ],
       [27.06566667],
       ...,
       [53.983     ],
       [44.87083333],
       [38.08666667]])

In [74]:
from sklearn.model_selection import train_test_split
X_train0_hour, X_test_hour, y_train0_hour, y_test_hour = train_test_split(X_hour, y_hour, random_state=42)
X_train1_hour, X_validation_hour, y_train1_hour, y_validation_hour = train_test_split(X_train0_hour, y_train0_hour, random_state=42)

In [75]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

X_train_s_hour=scaler.fit_transform(X_train1_hour)
X_validation_s_hour=scaler.fit_transform(X_validation_hour)
X_test_s_hour=scaler.fit_transform(X_test_hour)
y_train_s_hour=scaler.fit_transform(y_train1_hour)
y_validation_s_hour=scaler.fit_transform(y_validation_hour)
y_test_s_hour=scaler.fit_transform(y_test_hour)

In [76]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train_s_hour, y_train_s_hour, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [77]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.